In [1]:
import pandas as pd
import random

names = ['John', 'Emma', 'Michael', 'Olivia', 'William', 'Ava', 'James', 'Isabella', 'Benjamin', 'Mia']

In [27]:
# Get random sales
sales=[]
for _ in range(0,10,1):
    sales.append(random.randint(1000,10000))
col_names=["employee","sales","rank"]
outdf=pd.DataFrame()
outdf["names"]=names
outdf["sales"]=sales
outdf["rank"]=outdf["sales"].rank(method='min', ascending=False).astype(int)
outdf

,names,sales,rank
0,John,8824,2
1,Emma,9829,1
2,Michael,5789,4
3,Olivia,2501,9
4,William,3450,7
5,Ava,4403,6
6,James,5944,3
7,Isabella,1719,10
8,Benjamin,4806,5
9,Mia,2674,8


In [25]:
# Example data frame 生成最原始表
tsls=[]
for n in names:
    for y in range(2023,2025,1):
        for m in range(1,13,1):
            tsls.append([n,y,str(y)+"-"+str(m),random.randint(1000,10000)])
df=pd.DataFrame(data=tsls,columns=["staff_name","year","month","anp_php"])
df

,staff_name,year,month,anp_php
0,John,2023,2023-1,3172
1,John,2023,2023-2,2129
2,John,2023,2023-3,7778
3,John,2023,2023-4,2248
4,John,2023,2023-5,4929
...,...,...,...,...
235,Mia,2024,2024-8,1496
236,Mia,2024,2024-9,4493
237,Mia,2024,2024-10,8116
238,Mia,2024,2024-11,1224


In [51]:
# 分别生成根据年销售额排名的新表，year_rank_df
# 根据月销售额排名的信标，month_rank_df
# 1.将相关数据切片出来，不要影响原始表格
df_sliced=df[["staff_name","year","month","anp_php"]]
unique_years=df_sliced["year"].unique()
year_rank_df=pd.DataFrame()
month_rank_df=pd.DataFrame()
# 2.根据原始表格里面有的年和月份逐一循环生成年表和月表，为了防止有些人在，有些人可能离职的情况，不要用人去筛选和关联，只用作排名，有几个就排几个，有几个月就拍几个月，有几年就排几年
for y in unique_years:
    sub1_df=df_sliced.loc[(df["year"]==y)]
    year_sum_df = df_sliced.groupby('staff_name')['anp_php'].sum().reset_index() 
    year_sum_df["year_rank"]=year_sum_df["anp_php"].rank(method='min', ascending=False).astype(int)
    year_sum_df["year"]=y
    year_sum_df["anp_php_year"]=year_sum_df["anp_php"]
    year_sum_df.drop(columns=["anp_php"],inplace=True)
    year_rank_df=pd.concat([year_rank_df,year_sum_df], ignore_index=True)
    unique_months=sub1_df["month"].unique()
    for m in unique_months:
        sub2_df=df_sliced.loc[(df_sliced["year"]==y) & (df_sliced["month"]==m)]
        sub2_df["month_rank"]=sub2_df["anp_php"].rank(method='min', ascending=False).astype(int)
        month_rank_df=pd.concat([month_rank_df,sub2_df],ignore_index=True)
# 3生成完要核对
print(year_rank_df.head(10))
print(month_rank_df.head(10))

  staff_name  year_rank  year  anp_php_year
0        Ava          1  2023        151914
1   Benjamin          9  2023        114144
2       Emma          2  2023        140596
3   Isabella          6  2023        126209
4      James          3  2023        140059
5       John          4  2023        135975
6        Mia          8  2023        116234
7    Michael         10  2023        111363
8     Olivia          5  2023        129108
9    William          7  2023        121380
  staff_name  year   month  anp_php  month_rank
0       John  2023  2023-1     3172           7
1       Emma  2023  2023-1     9244           1
2    Michael  2023  2023-1     9166           2
3     Olivia  2023  2023-1     8068           3
4    William  2023  2023-1     4541           6
5        Ava  2023  2023-1     6799           4
6      James  2023  2023-1     1731           8
7   Isabella  2023  2023-1     1592           9
8   Benjamin  2023  2023-1     1241          10
9        Mia  2023  2023-1     4886 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10224\1535403463.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub2_df["month_rank"]=sub2_df["anp_php"].rank(method='min', ascending=False).astype(int)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10224\1535403463.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub2_df["month_rank"]=sub2_df["anp_php"].rank(method='min', ascending=False).astype(int)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10224\1535403463.py:20: SettingWithCop

In [52]:
#4 根据生成的新表，用原始表的索引做匹配，得到相关的新字段
def GetMonthRank(row,month_df):
    return month_df.loc[(month_df["year"]==row["year"])&(month_df["staff_name"]==row["staff_name"])&(month_df["month"]==row["month"])]["month_rank"]
df["month_rank"]=df.apply(lambda x:GetMonthRank(x,month_rank_df),axis=1)
df

ValueError: Columns must be same length as key